In [11]:
import pandas as pd
import numpy as np
import nltk
import gensim

from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet') 

# IImporting dataset
dataset = pd.read_csv("../data/AmazonData.csv")

[nltk_data] Downloading package punkt to /home/leo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/leo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/leo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
# Dataset process function
def dataset_process(dataset):

    # Excluding columns that we dont use
    cols = [0,2,3,5,6,8,9,12,14,15,16,17,18,19,20,21,22,23,24,25,26,27]
    dataset.drop(dataset.columns[cols], axis =1, inplace=True)
    dataset.dropna(inplace = True)

    # Splitting Category in 3 parts
    new = dataset["Category"].str.split("|", n = 3, expand = True)
    
    # making the first category called Main Category
    dataset["Main Category"]= new[0] 
    
    # making the second category called sub_category 
    dataset["Sub Category"]= new[1]

    # making the third category called side_category 
    dataset["Side Category"]= new[2]

    # making the last column consist of the remaining categories
    dataset["Other Category"]= new[3]

    # Dropping old category columns and the remaining categories 
    dataset.drop(columns =["Category"], inplace = True)

    # Setting Column Selling Price as float value
    # Database Price and weight treatment
    dataset.rename(columns = {'Uniq Id':'Id','Shipping Weight':'Shipping Weight(Pounds)', 'Selling Price':'Selling Price($)'}, inplace = True)

    # Removing units from Price and Weight
    dataset['Shipping Weight(Pounds)'] = dataset['Shipping Weight(Pounds)'].str.strip('ounces')
    dataset['Shipping Weight(Pounds)'] = dataset['Shipping Weight(Pounds)'].str.strip('pounds')
    dataset['Selling Price($)'] = dataset['Selling Price($)'].str.replace('$', '')

    # Removing rows with Total Price invalid
    indexes = dataset[dataset['Selling Price($)'] == 'Total price:'].index
    dataset.drop(indexes, inplace=True)

    # Removing rows with '-' character
    dataset['Selling Price($)'] = dataset['Selling Price($)'].str.replace(',', '', regex=False)
    indexes = dataset[dataset['Selling Price($)'].str.contains('-', na=False)].index
    dataset.drop(indexes, inplace=True)

    # Removing rows with '&' character
    indexes = dataset[dataset['Selling Price($)'].str.contains('&', na=False)].index
    dataset.drop(indexes, inplace=True)

    # Removing rows with 'Currently' character
    indexes = dataset[dataset['Selling Price($)'].str.contains('Currently', na=False)].index
    dataset.drop(indexes, inplace=True)

    # Removing rows with 'from' character
    indexes = dataset[dataset['Selling Price($)'].str.contains('from', na=False)].index
    dataset.drop(indexes, inplace=True)

    # Adjusting values with wrong format
    dataset['Selling Price($)'] = dataset['Selling Price($)'].str.split(' ').str[0]
    dataset['Selling Price($)'] = dataset['Selling Price($)'].astype(float)

    # Setting Column Shipping Weight as float value
    indexes = dataset[dataset['Shipping Weight(Pounds)'].str.contains(r'\. ', na=False)].index

    dataset.at[1619, 'Shipping Weight(Pounds)']
    dataset.drop(1619, inplace=True)
    dataset['Shipping Weight(Pounds)'] = dataset['Shipping Weight(Pounds)'].str.replace(',', '', regex=False)
    dataset['Shipping Weight(Pounds)'] = dataset['Shipping Weight(Pounds)'].astype(float)

    return dataset

In [13]:
# Processing text function
def preprocess_text(text):
    text = text.replace('[^a-zA-Z]',' ').lower()
    stop_re = '\\b'+'\\b|\\b'.join(nltk.corpus.stopwords.words('english'))+'\\b'
    text = text.replace(stop_re, '')
    text = text.split()

    # Add lemmatization using WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    lemmatized_text = [lemmatizer.lemmatize(word) for word in text]
    return lemmatized_text

In [14]:
# Creating a Word2Vec model with a vector size of 100, using the 'Processed Product Name' column.
def train_word2vec_model(dataset):
    model = Word2Vec(sentences=dataset["Processed Product Name"], vector_size=100, window=5, min_count=1, workers=4)
    return model

In [15]:
# vectorizing text function
def vectorize_product(product_name, model):

    words = [word for word in product_name if word in model.wv]
    if len(words) > 0:
        return np.mean([model.wv[word] for word in words], axis=0)
    else:
        return np.zeros(model.wv.vector_size)

In [16]:
# Product Recommendation function
def product_recommendation(product_vector, dataset, top_n=5):
    
    # Calcular similaridades cosseno
    similarities = dataset["Product Vector"].apply(lambda x: cosine_similarity([product_vector], [x])[0][0])
    
    # Ordenar por similaridade e pegar os top_n produtos
    top_indices = similarities.nlargest(top_n).index
    
    # Retornar o DataFrame com os produtos recomendados, mas mantendo os nomes originais
    return dataset.loc[top_indices, dataset.columns != 'Product Vector']

In [17]:
# Category Filter 
def category_filter(dataset, selected_product):

    main_category_input = selected_product['Main Category']
    sub_category_input = selected_product['Sub Category']
    side_category_input = selected_product['Side Category']
    other_category_input = selected_product['Other Category']

    # Create a new column to calculate the score
    dataset['score'] = 0

    # Raises the score if categories match
    dataset.loc[dataset['Main Category'] == main_category_input, 'score'] += 1
    dataset.loc[dataset['Sub Category'] == sub_category_input, 'score'] += 1
    dataset.loc[dataset['Side Category'] == side_category_input, 'score'] += 1
    dataset.loc[dataset['Other Category'] == other_category_input, 'score'] += 1

    # Sort the database based on the score
    category_filter = dataset.sort_values(by='score', ascending=False)

    # Remove products with score < 1
    category_filter = category_filter[category_filter['score'] > 1]

    # Removes the new column
    category_filter = category_filter.drop(columns='score')

    # return the sorted database
    if category_filter.empty:
        print('No recommendaation found for this product.')
    else:
        return category_filter

In [18]:

def name_based_filter(dataset, product_name):

    # Applying text preprocess in dataset
    dataset["Processed Product Name"] = dataset["Product Name"].apply(preprocess_text)

    model = train_word2vec_model(dataset)

    # Applying vectorizing function in dataset
    dataset["Product Vector"] = dataset["Processed Product Name"].apply(lambda x: vectorize_product(x, model))

    # Pré-processando o nome do produto fornecido pelo usuário
    processed_product_name = preprocess_text(product_name)

    # Vetorizando o nome do produto fornecido (passar o modelo aqui também)
    product_vector = vectorize_product(processed_product_name, model)

    recommendation = product_recommendation(product_vector, dataset, top_n=5)
    return recommendation

In [19]:
dataset = dataset_process(dataset)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7136 entries, 0 to 10001
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Product Name             7136 non-null   object 
 1   Selling Price($)         7136 non-null   float64
 2   About Product            7136 non-null   object 
 3   Product Specification    7136 non-null   object 
 4   Shipping Weight(Pounds)  7136 non-null   float64
 5   Main Category            7136 non-null   object 
 6   Sub Category             7136 non-null   object 
 7   Side Category            6155 non-null   object 
 8   Other Category           2724 non-null   object 
dtypes: float64(2), object(7)
memory usage: 557.5+ KB


/home/leo/anaconda3/envs/tcc/lib/python3.7/site-packages/ipykernel_launcher.py:34: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


In [20]:
print("Please select option:")
print("1- Select input from database")
print("2- Type product name")
user_input = int(input("Enter 1 or 2: "))

main_category_input = None
sub_category_input = None
side_category_input = None
other_category_input = None

if user_input == 1:
    product_line = int(input("Type the product's number: "))
    product_line = product_line - 2

    if 0 <= product_line < len(dataset):
        selected_product = dataset.iloc[product_line]  # Selects the product by its line in the database
        print("\nSelected Product:")
        print(selected_product)
    else:
        print("Invalid line Number.")
    
    product_name = selected_product['Product Name']

    category_filter = category_filter(dataset, selected_product)
    # print(category_filter)

    name_based_filter = name_based_filter(category_filter, product_name)
    print(name_based_filter)
    
elif user_input == 2:
    product_name = input("Type the product's name: ")
    name_based_filter = name_based_filter(dataset, product_name)
    print(name_based_filter)

else:
    print("Not a valid number")

print("\n")
print(product_name)
# Checking if categories are valid
if main_category_input not in dataset['Main Category'].unique():
    print('Main Category not found in the database.')
else:
    print(main_category_input)
if sub_category_input not in dataset['Sub Category'].unique():
    print('Subcategoria not found in the database.')
else:
    print(sub_category_input)
if side_category_input not in dataset['Side Category'].unique():
    print('Side Category not found in the database.')
else:
    print(side_category_input)
if other_category_input not in dataset['Other Category'].unique():
    print('Other Category not found in the database.')
else:
    print(other_category_input)

Please select option:
1- Select input from database
2- Type product name



Selected Product:
Product Name               LEGO Minecraft Creeper BigFig and Ocelot Chara...
Selling Price($)                                                       14.99
About Product              Make sure this fits by entering your model num...
Product Specification      ProductDimensions:5.5x7.5x1.8inches|ItemWeight...
Shipping Weight(Pounds)                                                  5.6
Main Category                                                  Toys & Games 
Sub Category                                                  Building Toys 
Side Category                                                  Building Sets
Other Category                                                          None
Name: 13, dtype: object
                                           Product Name  Selling Price($)  \
13    LEGO Minecraft Creeper BigFig and Ocelot Chara...             14.99   
6051  LEGO Minecraft The Panda Nursery 21158 Constru...             19.99   
920   LEGO Star Wars Resistance Y